# Memory and resource deallocation test.

This notebook uses the NER example code to test for resource leaks.  Please see the `ner.ipynb` for the actual NER notebook example.

In [ ]:
class NotebookHarness(object):
    """Configure the Jupyter notebook environment and create model resources."""
    def __init__(self, app_root_dir: str = '..', deepnlp_path: str = '../../../src/python'):
        """Set up the interpreter environment so we can import local packages.
        
        :param app_root_dir: the application root directory
        :param deepnlp_path: the path to the DeepNLP source code
        """
        import sys
        from pathlib import Path
        self.app_root_dir = Path(app_root_dir)
        # add the example to the Python library path
        sys.path.append(str(self.app_root_dir / 'src'))
        # add the deepnlp path
        sys.path.append(deepnlp_path)
        # reset random state for consistency before any other packages are imported
        from zensols.deepnlp import init as nlp_init
        nlp_init()

    def __call__(self, cuda_device_index: int = None, temporary_dir_name: str = None):
        """Create and return an instance a :class:`.JupyterManager`.
        
        :param cuda_device_index: the CUDA (GPU) device to use
        :param temporary_dir_name: the temporary directory to use for temporary space and results
        """
        from zensols.deeplearn.cli import JupyterManager
        from movie import CliFactory
        factory_args = {'root_dir': self.app_root_dir}
        if temporary_dir_name is not None:
            factory_args['temporary_dir'] = self.app_root_dir / temporary_dir_name
        mng = JupyterManager(
            allocation_tracking=True,
            cli_class=CliFactory,
            factory_args=factory_args,
            cli_args_fn=lambda model: ['-c', str(self.app_root_dir / 'models' / f'{model}.conf')])
        if cuda_device_index is not None:
            # tell which GPU to use
            mng.config('gpu_torch_config', cuda_device_index=cuda_device_index)
        return mng


harness = NotebookHarness()
mng = harness()
# number of epochs to test with, which needs to be at least two to excercise it properly
mng.config('model_settings', epochs=2)
# if a memory leak test fails, set to False for that test
keep_going = True

In [ ]:
from io import StringIO
facade = mng.create_facade('glove50')
# write to force more allocation to test deallocation, but don't clutter the notebook output
facade.write(writer=StringIO())
mng.run(False)

In [ ]:
if keep_going:
    facade.embedding = 'glove_300_embedding'
    mng.run(False)
    mng.show_leaks()

In [ ]:
if keep_going:
    facade = mng.create_facade('transformer-fixed')
    mng.run(False)

In [ ]:
if keep_going:
    facade = mng.create_facade('transformer-trainable')
    mng.run(False)
    mng.show_leaks()